In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

catalog_name = 'opensky'

In [0]:
df = spark.table(f'{catalog_name}.default.raw')


In [0]:
display(df.limit(5))

In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import col, to_timestamp, from_unixtime

df = df.withColumn('icao24',col('icao24').cast(StringType()))\
       .withColumn('callsign',col('callsign').cast(StringType()))\
       .withColumn("time_position", from_unixtime(col("time_position")).cast("timestamp")) \
       .withColumn("last_contact", from_unixtime(col("last_contact")).cast("timestamp")) \
       .withColumn("longitude", col("longitude").cast("float")) \
       .withColumn("latitude", col("latitude").cast("float")) \
       .withColumn("ingest_time", from_unixtime(col("ingest_time")).cast("timestamp")) \
       .withColumn("geo_altitude", col("geo_altitude").cast("float")) \
       .withColumn("velocity", col("velocity").cast("float")) \
       .withColumn("heading", col("heading").cast("float")) \
       .withColumn("vertical_rate", col("vertical_rate").cast("float")) \
       .withColumn("baro_altitude", col("baro_altitude").cast("float"))\
       .withColumn("on_ground", col("on_ground").cast("boolean"))\
       .withColumn("spi", col("spi").cast("boolean")).withColumn('squawk',col('squawk').cast(IntegerType()))\
       .withColumn('origin_country',col('origin_country').cast(StringType()))\
       .withColumn("position_source", col("position_source").cast("int"))


## Generate an html report about data quality

In [0]:
%pip install ydata_profiling

In [0]:
dbutils.library.restartPython()

In [0]:
from ydata_profiling import ProfileReport
# data profiling with 20% of the dataset

df_20 = df.sample(fraction=0.2, seed=42).toPandas()
profile = ProfileReport(df_20, title="My Dataset")
profile.to_file("report.html")

## Data columns meaning 
* icao24: Aircraft unique identifier 
* callsign: Flight’s spoken identifier; can be null
* origin_country: Country where the aircraft is registered or operated from
* time_position: Unix timestamp of last position update; can be null
* last_contact: Unix timestamp of last signal received from the aircraft
* longitude: Longitude of the aircraft’s position; can be null
* latitude: Latitude of the aircraft’s position; can be null
* geo_altitude: Geometric altitude in meters; can be null
* on_ground: Indicates if the aircraft is on the ground
* velocity: Ground speed in meters per second; can be null
* heading: True track (heading) over ground in degrees (0–359); can be null
* vertical_rate: Vertical rate in meters per second; can be null
* baro_altitude: Barometric altitude in meters; can be null
* squawk: Transponder squawk code (4-digit code for radar identification); can be null
* spi: Special Purpose Indicator (SPI) or IDENT flag; true if set, false otherwise
* position_source: Source of position info (e.g., ADS-B, MLAT); can be null
* ingest_time: Timestamp when data was ingested


## Data rules check

In [0]:

from pyspark.sql.functions import col, when, lit, abs


df_qc = df.withColumn(
    "dq_status",
    when(
        # CORRUPTED → impossible or broken identifiers
        ( (col("icao24").isNull()) | ~col("icao24").rlike("^[0-9A-Fa-f]{6}$")) |
        (col("latitude") < -90) | (col("latitude") > 90) |
        (col("longitude") < -180) | (col("longitude") > 180) |
        (col("heading") < 0) | (col("heading") >= 360) |
        (~col("position_source").isin(0,1,2)), # o value refer to the Automatic Dependent Surveillance-Broadcast (ADS-B) source
        lit("CORRUPTED")
    )
    .when(
        # NOISY → sensor noise but still usable
        ((col("geo_altitude").isNull()) | (col("geo_altitude") < -50 )) |

        ((col("baro_altitude").isNull()) | (col("baro_altitude") < -50 )) |

        # (abs(col("geo_altitude") - col("baro_altitude")) > 2000) |

        ( (col("time_position").isNull()) | (col("time_position") > current_timestamp()) ) |

        ( (col("last_contact").isNull()) | (col("last_contact") > current_timestamp()) ) |
        
        ( (col("ingest_time").isNull()) | (col("ingest_time") > current_timestamp()) ) |

        ((col("on_ground").isNull() | (col("on_ground") == True)) & col("velocity").isNotNull() & (col("velocity") > 10)) | # Cannot be on ground and velocity > 10 m/s | if on_ground_isnull make declare it as noisy 

        ((col("on_ground") == False) & (col("velocity").isNotNull()) & (col("velocity") < 26)) | # Cannot be on air and velocity < 26 m/s

        ((col("on_ground") == True) & (col("vertical_rate").isNotNull()) & (col("vertical_rate") != 0 )) # cannnot be on ground and vertical rate is not equal to zero
        

        ,lit("NOISY")
    )
    .otherwise(lit("VALID"))
)

# Auto clean sensor noise 
df_clean = df_qc \
    .withColumn("geo_altitude",
        when(col("geo_altitude") < -50, None).otherwise(col("geo_altitude"))
    ) \
    .withColumn("baro_altitude",
        when(col("baro_altitude") < -50, None).otherwise(col("baro_altitude"))
    ) \
    .withColumn("velocity",
        when(col("velocity") < 0, None).otherwise(col("velocity"))
    ) \



In [0]:
# Data quality report 
dq_report = df_clean.groupBy("dq_status").count()

display(dq_report)

# we Can drop noisy data as it doesnt form a big pourcentage for the overal clean data we can still make use of it in further reporting cases

In [0]:
# Column-level  violation counters 
from pyspark.sql.functions import sum 

violations = df.select(
    sum((~col("icao24").rlike("^[0-9A-Fa-f]{6}$")).cast("int")).alias("bad_icao24"),
    sum(((col("latitude") < -90) | (col("latitude") > 90)).cast("int")).alias("bad_lat"),
    sum(((col("longitude") < -180) | (col("longitude") > 180)).cast("int")).alias("bad_lon"),
    sum((col("geo_altitude") < -50).cast("int")).alias("bad_geo_alt"),
    sum((col("velocity") < 0).cast("int")).alias("bad_velocity"),
    sum((~col("position_source").isin(0,1,2)).cast("int")).alias("bad_position_source")
)

violations.show()


In [0]:
df.select('position_source').distinct().show()
'only availabe value is 0 : thats refers to teh Automatic Dependent Surveillance-Broadcast (ADS-B) system'

In [0]:
# 100 country
df.select('origin_country').distinct().show()